# Cosmeet 商品情報スクレイピング

このノートブックは [cosmeet.cosme.net](https://cosmeet.cosme.net) から商品情報を取得します。

- 使用技術：Scrapy + Selenium
- 対象：動的ページに対応（JS描画あり）
- 出力：output.csv
- ページ数：10ページ分のデータ取得

In [71]:
!pip install scrapy selenium webdriver-manager pandas


[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


In [72]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
from scrapy.http import HtmlResponse
import platform
import pandas as pd

In [73]:
def scrape_cosmeet(pages=1):
    # Chromeオプション設定（macOS対応）
    chrome_options = Options()
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument("--disable-dev-shm-usage")

    # macOS (Apple Silicon) の場合 headless=new を使用
    if platform.system() == "Darwin":
        chrome_options.add_argument("--headless=new")
    else:
        chrome_options.add_argument("--headless")

    # ✅ ChromeDriver を起動（バージョン指定は不要）
    driver = webdriver.Chrome(
        service=Service(ChromeDriverManager().install()),
        options=chrome_options
    )

    base_url = "https://cosmeet.cosme.net/product/search?page={}"
    results = []

    for page in range(pages):
        url = base_url.format(page)
        print(f"🔗 アクセス: {url}")
        driver.get(url)
        WebDriverWait(driver, 3).until(
            EC.presence_of_all_elements_located((By.CSS_SELECTOR, "div.mdl-pdt-idv.clearfix"))
        )

        response = HtmlResponse(url=driver.current_url, body=driver.page_source.encode("utf-8"), encoding="utf-8")
        products = response.css("div.mdl-pdt-idv.clearfix")
        print(f"🟢 商品数: {len(products)}")

        for p in products:
            item = {
                "product_url": response.urljoin(p.css("a::attr(href)").get()),
                "name": p.css("img::attr(alt)").get(default="").strip(),
                "brand": "", "rating": "", "category": "", "description": "",
                "size_price": "", "release_date": "", "product_id": "",
                "maker": "", "features": "", "want_count": "", "have_count": ""
            }

            # 詳細ページへ遷移
            driver.get(item["product_url"])
            WebDriverWait(driver, 3).until(
                EC.presence_of_all_elements_located((By.CSS_SELECTOR, "body"))
            )
            detail = HtmlResponse(url=driver.current_url, body=driver.page_source.encode("utf-8"), encoding="utf-8")

            # 各種情報の抽出
            item["product_id"] = item["product_url"].split("/")[-2]
            item["brand"] = detail.css("span.brd-name a::text").get(default="").strip()
            item["rating"] = detail.css("p.average::text").get(default="").strip()
            item["category"] = " ".join([c.strip() for c in detail.css("dl.item-category a::text").getall()])
            item["description"] = detail.css("dl.item-description.clearfix dd::text").get(default="").strip()
            item["size_price"] = detail.css("p.info-desc::text").get(default="").strip()
            release = detail.xpath('//p[contains(text(), "発売日")]/following-sibling::p[@class="info-desc"]/text()').get()
            item["release_date"] = release.strip() if release else ""
            item["maker"] = detail.css("dl.maker.clearfix a::text").get(default="").strip()
            item["features"] = " ".join([f.strip() for f in detail.css("dl.ingredient a::text").getall()])
            item["want_count"] = detail.xpath('//div[@class="act-counter" and contains(@data-object.class, "product_like")]/text()').get(default="").strip()
            item["have_count"] = detail.xpath('//div[@class="act-counter" and contains(@data-object.class, "product_have")]/text()').get(default="").strip()

            results.append(item)

    driver.quit()
    return results

In [74]:
results = scrape_cosmeet(pages=1)
print(f"✅ 取得件数: {len(results)}")

🔗 アクセス: https://cosmeet.cosme.net/product/search?page=0
🟢 商品数: 10
✅ 取得件数: 10


In [75]:
df = pd.DataFrame(results)
df.to_csv("output.csv", index=False)
print("✅ CSVファイルに保存完了: output.csv")

✅ CSVファイルに保存完了: output.csv


In [76]:
df.head(20)

,product_url,name,brand,rating,category,description,size_price,release_date,product_id,maker,features,want_count,have_count
0,https://www.cosme.net/products/10245099/,トランスルーセントセットフィニッシングパウダー/tfit,tfit,5.5,ベースメイク フェイスパウダー ルースパウダー,長時間「アプリ加工肌」続く*,"7g・1,760円",2023/9/15,10245099,TFIT CLASS,? ヒアルロン酸 アレルギーテスト済,"4,888","8,588"
1,https://www.cosme.net/products/10232368/,AGE-Rスキンブースターショット/MEDICUBE(メディキューブ),MEDICUBE(メディキューブ),6.2,美容グッズ・美容家電 スキンケアグッズ スキンケア美容家電,手持ちの,"30,800円",-,10232368,APR JAPAN,,"1,530",246
2,https://www.cosme.net/products/10271786/,コスメデコルテ ＡＱ 毛穴美容液オイル/コスメデコルテ,コスメデコルテ,5.4,スキンケア・基礎化粧品 クレンジング オイルクレンジング スキンケア・基礎化粧品 洗顔料 そ...,角栓を溶かし崩し、輝きあふれる,"40mL・11,000円",2025/5/16,10271786,コスメデコルテ,? 無鉱物油 アルコールフリー パラベンフリー アレルギーテスト済,"9,644","4,813"
3,https://www.cosme.net/products/10263025/,スーパーラメラシャンプー&amp;EXモイストトリートメント ＦＯＲ ＤＡＩＬＹ ＤＡＭＡＧ...,THE ANSWER,5.1,ヘアケア・スタイリング シャンプー・コンディショナー シャンプー・コンディショナー,■一部バラエティストア、,8g / 12ml / 220g / 320ml / 400ml (オープン価格),2024/11/2,10263025,花王,? コラーゲン ヒアルロン酸 AHA(フルーツ酸),"13,299","17,627"
4,https://www.cosme.net/products/10254107/,フィー 3Dボリューミンググロス/fwee(フィー),fwee(フィー),5.4,メイクアップ 口紅・グロス・リップライナー リップグロス,同じカラーを濃度別に30％/70%の２パターン展開、なりたい雰囲気に合わせてチョイス。(全1...,"5.3g・1,760円",2024/7/1 (2025/3/28追加発売),10254107,BENOW,,"14,789","36,091"
5,https://www.cosme.net/products/10252345/,AGE-Rブースタープロ/MEDICUBE(メディキューブ),MEDICUBE(メディキューブ),6.0,美容グッズ・美容家電 スキンケアグッズ スキンケア美容家電,1台6役で実現できるトータルケア,"34,001円",2024/2/7 (2024/10/25追加発売),10252345,APR JAPAN,,"5,833",418
6,https://www.cosme.net/products/10234175/,サンシェルター マルチ プロテクション トーンアップCC/コスメデコルテ,コスメデコルテ,5.3,ベースメイク 化粧下地・コンシーラー 化粧下地 日焼け対策・UVケア 日焼け止め・UVケア(顔用),SPF50+・PA++++,"35g・3,520円",2023/2/16,10234175,コスメデコルテ,,"80,101","123,843"
7,https://www.cosme.net/products/10259593/,PDRNピンクアンプル/MEDICUBE(メディキューブ),MEDICUBE(メディキューブ),5.2,スキンケア・基礎化粧品 乳液・美容液など 美容液,肌が食べる,"2,530円",2024/7/10,10259593,APR JAPAN,,"6,016","5,056"
8,https://www.cosme.net/products/10224408/,AGE-Rダーマエアショット/MEDICUBE(メディキューブ),MEDICUBE(メディキューブ),5.1,美容グッズ・美容家電 スキンケアグッズ スキンケア美容家電,「AGE-Rダーマエアショット」はダーマペンの原理を利用した家庭用,"29,700円",2022/4/9,10224408,APR JAPAN,,"1,402",279
9,https://www.cosme.net/products/10263023/,PDRN ヒアルロン酸100 セラム/Anua,Anua,5.3,スキンケア・基礎化粧品 乳液・美容液など 美容液 スキンケア・基礎化粧品 化粧水 ブースター...,「PDRN、,"1ml・オープン価格 / 30ml・2,970円",2024/12/16,10263023,The Founders,? コラーゲン ヒアルロン酸 無香料,"11,262","12,604"
